In [ ]:
df.info()
#Helps check for null and none number values 
#Drop columns
drg = drg.drop(columns=["n", 'alcohol-frequency'])
dfE.drop(columns={"BodyFat"}, inplace=True)
#Rename column for teams
hock = hock.rename(columns={'Unnamed: 1':"Team"})

hw = hw.dropna(axis=0)

#Are there nulls?
df.isnull().sum().sort_values()

In [ ]:
#Create a scatterplot. 
thinkplot.Scatter(df["height"], df["weight"])
thinkplot.Show(xlabel="Height", ylabel="Weight", axis=[120,220,20,160])

In [ ]:
#Add jitter. 
thinkplot.Scatter(thinkstats2.Jitter(df["height"]), thinkstats2.Jitter(df["weight"]), alpha=.3)
thinkplot.Show(xlabel="Height", ylabel="Weight", axis=[120,220,20,160])

In [ ]:
#Calculate both correlation metrics
a=df["height"].values.tolist()
b=df["weight"].values.tolist()
thinkstats2.Corr(a, b), thinkstats2.SpearmanCorr(a,b)

In [ ]:
#Cut down data to only numeric values, and fliter out some outliers. 
df2 = df[["age", "gender", "height", "weight", "ap_lo", "ap_hi"]]
df2 = df2[(df2["height"]>110) & (df2["height"] < 200)]

In [ ]:
#Generate pairplot
sns.pairplot(df2,kind="reg", dropna=True, hue="gender")
plt.show()

In [ ]:
#Take the data in our dataframe, and change it into a set of correlations. 
drg2 = drg.apply(pd.to_numeric, errors='coerce')
drg2 = drg2.drop(columns=["age"])
drg2 = drg2.corr()
drg2.head()

In [ ]:
#Show Correlations in heatmap
mask = np.triu(np.ones_like(drg2, dtype=bool))
sns.heatmap(drg2, center=0, linewidths=.5, annot=True, cmap="YlGnBu", yticklabels=True, mask=mask)
plt.show()

In [ ]:
sns.displot(data=d, x="Glucose", kde=True, col="Outcome", stat="density")

In [ ]:
#Non-Diabetic people are normal!
d_neg = d[d["Outcome"]==0]
thinkstats2.NormalProbabilityPlot(d_neg["Glucose"])



#Create a normal distribution. Because we have stated "it is normal",
norm = ss.norm(loc=d_neg["Glucose"].mean(), scale=d_neg["Glucose"].std())


#Skewnorm is another distribution - a normal one with some skew. We calculate it, and pass it to our distribution with a=
a = thinkstats2.PearsonMedianSkewness(d_neg["Glucose"])
norms = ss.skewnorm(loc=d_neg["Glucose"].mean(), scale=d_neg["Glucose"].std(), a=a)
norms.cdf(6), norms.median(), norms.mean(), a

#Create a model that is fitted to the data, automagically.
ae, loce, scalee = ss.skewnorm.fit(d_neg["Glucose"])
snormFit = ss.skewnorm(ae, loce, scalee)

#Plot all the distributions together. 
sns.distplot(norms.rvs(size=100000), color="red")
sns.distplot(norm.rvs(size=100000), color="blue")
sns.distplot(snormFit.rvs(size=100000), color="yellow")
sns.distplot(d_neg["Glucose"], color="green") #This is the actual data
plt.show()


In [ ]:
#Generate regression plot 
slope, intercept, r_value, pv, se = ss.linregress(drg["alcohol-use"], drg["heroin-use"])
print("Slope:", slope)
print("Intercept:", intercept)
equation = "Equation:  y=", slope, "* x +", intercept
#print(equation)

sns.regplot(data=drg, x="alcohol-use", y="heroin-use", dropna=True, ci=0).set(title=equation)

In [ ]:
#Add old predictions
#Note: to figure out the command I Googled "dataframe moving average" and got the rolling function
df["avg5"] = df["Price"].rolling(5, closed="left").mean()
#Calculate the residuals (errors) and put them into the table
df["res5"] = df["Price"]-df["avg5"]



#Calculate RMSE using prebuilt functions. Hopefully the results are the same!!
from sklearn.metrics import mean_squared_error
# real value
expected = df["Price"]
#Note: The iloc thing selects the rows. The function can't deal with missing values.
# predicted value
predicted5 = df["avg5"].iloc[5:]
predicted3 = df["avg3"].iloc[3:]
# calculate errors. Change squared to True to get MSE
errors5 = mean_squared_error(expected.iloc[5:], predicted5, squared=False)
errors3 = mean_squared_error(expected.iloc[3:], predicted3, squared=False)
errors5, errors3

# Add Residuals
d2["residual"] = d2["heroin-use"]-d2["pred"]
#RMS  and RMSE
d2["res2"] = d2["residual"]**2
m2e = d2["res2"].mean()
rm2e = d2["res2"].mean()**(1/2)

In [ ]:
def MeanError(estimates, actual):
    errors = [estimate-actual for estimate in estimates]
    return np.mean(errors)

In [ ]:
def RMSE(estimates, actual):
    e2 = [(estimate-actual)**2 for estimate in estimates]
    mse = np.mean(e2)
    return np.sqrt(mse)

In [ ]:
#Calculate RMSE from residuals
def rmseFromResiduals(residuals):
    SSE = 0
    for i in range(len(residuals)):
        SSE += (residuals[i]**2)
    MSE = SSE/len(residuals)
    return math.sqrt(MSE)

In [ ]:
#Calculate the RMSE, here using the statsmodel function. 
ypred = est2.predict(X2)
from statsmodels.tools.eval_measures import rmse
rmse(d[y1], ypred)

In [ ]:
#Sklearn RMSE
mean_squared_error(d[y1], ypred, squared=False)

In [ ]:
def normalEstimates(mu=0, median=0, sigma=1, n=1000, iters=1000):
    estimates1 = []
    estimates2 = []
    means = []
    medians = []
    for _ in range(iters):
        xs = np.random.normal(mu, sigma, n)
        xbar = np.mean(xs)
        means.append(xbar)
        xmed = np.median(xs)
        medians.append(xmed)
        biased = np.var(xs)
        unbiased = np.var(xs, ddof=1)
        estimates1.append(biased)
        estimates2.append(unbiased)

In [ ]:
#Normal Simulator
def simulateNormal(mu=0, sigma=1, n=100, m=10000, ciLow=5, ciHi=95):
    means = [] #List of all the means that are created. 
    preds = []
    for j in range(m): #Run m number of simulations. 
        xs = np.random.normal(mu, sigma, n) #Generate a normal dist based on emperical data. 
        xbar = np.mean(xs) #Take the mean of the dist above's values
        means.append(xbar) #Add to list. 
        preds.append(xs)
    cdf = thinkstats2.Cdf(means) #Make a CDF of the means of the analytical dist's
    ci = cdf.Percentile(ciLow), cdf.Percentile(ciHi) #5th, 95th percentiles. 
    muList = [mu] * m
    stderr = mean_squared_error(means, muList, squared=False) #RMSE of how different the random analytical means are from the emp. mean. 
    return cdf, ci, stderr, means, preds

In [ ]:
#Produce estimates - change the n and trials and observe results
mu = df["log.annual.inc"].mean()
sig = df["log.annual.inc"].std()
n = df["log.annual.inc"].count()
med = df["log.annual.inc"].median()
print("Mean:",mu," Median:",med)

cdf, ci, stderr, means, preds = simulateNormal(mu=med, sigma=sig, n=20, m=20, ciLow=5, ciHi=95)
print("Standard Error:", stderr)
print("Low CI:", ci[0])
print("Hi CI:", ci[1])
print("Mean of Means:", cdf.Mean())
thinkplot.Cdf(cdf)
thinkplot.axvline(cdf.Mean(), color="green")
thinkplot.axvline(ci[0], color="red")
thinkplot.axvline(ci[1], color="red")
thinkplot.Config()

In [ ]:
#Exponential simulator
def simulateExp(mean=1, n=100, m=10000, ciLow=5, ciHi=95):
    means = []
    for j in range(m):
        xs = np.random.exponential(1/mean,n)
        xbar = 1/np.mean(xs)
        means.append(xbar)
    cdf = thinkstats2.Cdf(means)
    ci = cdf.Percentile(ciLow), cdf.Percentile(ciHi)
    rateList = [mean] * m
    stderr = mean_squared_error(means, rateList, squared=False)
    return cdf, ci, stderr

In [ ]:
#plot em
sns.histplot(dPos, kde=True, stat="density")
sns.histplot(dNeg, kde=True, stat="density", color='red')

##### Simulate one Game

In [ ]:
def SimGame(lam):
    dist = ss.poisson(lam)
    score = dist.rvs(1)
    return score[0]

##### Simulate Many Games


In [ ]:
def estimateGame(lam=2., m=1000):
    estimates = []
    for i in range(m):
        L = SimGame(lam)
        estimates.append(L)

    print('Goal Distribution:')
    print('rmse Goals:', RMSE(estimates, lam))
    print('mean error Goals:', MeanError(estimates, lam))
    
    pmf = thinkstats2.Pmf(estimates)
    thinkplot.Hist(pmf)
    thinkplot.Config(xlabel='Goals scored', ylabel='PMF')

##### Create Winners

In [ ]:
#Estimate a bunch of games, count how many one team wins vs the other. 
def estimateMatch(team1=1, team2=1, m=1000):
    team1Wins = []
    ties = []
    for i in range(m):
        goal1 = SimGame(team1)
        goal2 = SimGame(team2)
        team1Wins.append(int(goal1>goal2))
        ties.append(int(goal1 == goal2))
    return team1Wins, ties

In [ ]:
# Print results
oil_win, oil_ties = estimateMatch(gfOil,gfFla,1000)
np.mean(oil_win), np.mean(oil_ties)

Election

In [ ]:
#How likely is it for someone to vote Trump and Biden?
probT = .46/(.49+.46)
print("Trump Prob:", probT)

probB = .49/(.49+.46)
print("Biden Prob:", probB)

In [ ]:
def oneVote(probCalc):
    vote = np.random.binomial(n=1, p=probCalc)
    return vote

In [ ]:
def getSample(voteProb, n=1000):
    vote_list = []
    for i in range(n):
        vote_list.append(oneVote(voteProb))
    return vote_list

In [ ]:
tmp = getSample(probB, 5000)
np.mean(tmp)

#### Do Some Simulating

In [ ]:
def getSamples(voteProb, n=1000, samples=100, ciLow=2.5, ciHi=97.5):
    meanList = []
    for i in range(samples):
        meanList.append(np.mean(getSample(voteProb, n)))
    muList = [voteProb] * samples
    cdf = thinkstats2.Cdf(meanList) #Make a CDF of the means of the analytical dist's
    ci = cdf.Percentile(ciLow), cdf.Percentile(ciHi) #5th, 95th percentiles. 
    stderr = mean_squared_error(meanList, muList, squared=False)
    return meanList, stderr, cdf, ci

### Predict the Winner!!!!

In [ ]:
#Run a bunch of trials and count how many Biden wins in. 
trials = 1000
means, err, cdfFin, ciFin = getSamples(probB, n=700, samples=trials)
lowest = means - err
highest = means + err
bWins = 0 
for i in range(len(means)):
    if means[i] > .5000:
        bWins = bWins + 1
print(bWins/trials)

In [ ]:
thinkplot.Cdf(cdfFin)
thinkplot.axvline(ciFin[0], color="red")
thinkplot.axvline(ciFin[1], color="red")
print("Fraction of Scenarios where Biden wins is %.1f%% " % ((1-cdfFin.Prob(.50))*100))
print("We are 95 percent confident that Biden will get between %.1f%% and %.1f%% percent of the vote" % (ciFin[0]*100, ciFin[1]*100))

### Run T-Test

In [ ]:
stat, pval = ss.ttest_ind(dPos, dNeg, alternative="greater")
pval

# Force Permutations
stat2, pval2 = ss.ttest_ind(dPosBP, dNegBP, permutations=100)
stat2, pval2

# mw
print(ss.mannwhitneyu(fv, boost))

#Check for signifigance

In [ ]:
cutoff = .05
if pval < cutoff:
    print("Reject null hypothesis - Effect appears significant")
else:
    print("Accept null hypothesis - Effect may be due to random chance")

Run the test by hand 

In [ ]:
dPG = dfP.BloodPressure
dNPG = dfNP.BloodPressure

gP, gNP = dPG.mean(), dNPG.mean()
print(gP, gNP)
gDiff = abs(gP - gNP)
print(gDiff, '<---That is my test stat. There is a diff in BP of this much, does it matter or is this difference likely to be random chance?')


In [ ]:
#Combines the two groups.
#Get the length of one for splitting below. 
allG = np.hstack((dPG, dNPG))
nP = len(dNPG)

#Generate random splits, and add the differnce in their means to a list. 
diffsP = []
#Write the loop. For each of 100 trials, shuffle the list of BP, then split it into two. 
for i in range(100):
    np.random.shuffle(allG)
    data1 = allG[nP:]
    data2 = allG[:nP]
    tmpDiff = abs(data1.mean()-data2.mean())
    diffsP.append(tmpDiff)
diffsP[0:5], "<--- Some Example Differences"

In [ ]:
#Take the differences in means from the random samples,
#Count how many times that difference exceeds our test stat. 
successP = 0
#Write the loop. Just like the previous one. 
for i in range(len(diffsP)):
    if diffsP[i] > gDiff:
        successP += 1
pP = successP/len(diffsP)
print("P =",pP)
#Check muh Null Hyp!!
cutoffP = .05
if pP < cutoffP:
    print("Reject null hypothesis - Effect appears significant")
else:
    print("Accept null hypothesis - Effect may be due to random chance")

### Test Power

In [ ]:
def CohenEffectSize(group1, group2):
    """Compute Cohen's d.

    group1: Series or NumPy array
    group2: Series or NumPy array

    returns: float
    """
    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()

    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    d = diff / np.sqrt(pooled_var)
    return d

In [ ]:
#Statsmodels calculation of power. This also indicates that we are making a type 2 error as well 
from statsmodels.stats.power import TTestPower
powerTest = TTestPower()
ces = thinkstats2.CohenEffectSize(fAge, mAge)
alpha = .05
nobs = mCount + fCount #number of observations 

pow = powerTest.power(effect_size=ces, nobs=nobs, alpha=alpha)
pow, ces


Calculate sample size

In [ ]:
#Calculate n
n = powerTest.solve_power(ces, power=.8, nobs=None, alpha=alpha)
n

## What About Categorical Data - Chi-2 Tests

In [ ]:
expected = [10,10,10,10,10,10]
observed = [8,9,19,5,8,11]
ss.chisquare(observed, expected)

## What if There's 3 or More Samples? 

In [ ]:
sns.boxplot(data=df2, x="Educ", y="Age")

In [ ]:
varStat, varP = ss.levene(df2['Age'][df2['Educ'] == 1],
               df2['Age'][df2['Educ'] == 2],
               df2['Age'][df2['Educ'] == 3],
               df2['Age'][df2['Educ'] == 4],
               df2['Age'][df2['Educ'] == 5])
varP

In [ ]:
print(np.var(df2['Age'][df2['SES'] == 1.0]))
print(np.var(df2['Age'][df2['SES'] == 2.0]))
print(np.var(df2['Age'][df2['SES'] == 3.0]))
print(np.var(df2['Age'][df2['SES'] == 4.0]))
print(np.var(df2['Age'][df2['SES'] == 5.0]))

In [ ]:
ss.f_oneway(df2['Age'][df2['Educ'] == 1],
               df2['Age'][df2['Educ'] == 2],
               df2['Age'][df2['Educ'] == 3],
               df2['Age'][df2['Educ'] == 4],
               df2['Age'][df2['Educ'] == 5])

## What if the Data Isn't Normal?

In [ ]:
sns.kdeplot(data=df2, x="CDR", hue="M/F")

In [ ]:
cdr_m = df2['CDR'][df2['M/F'] == "M"]
cdr_f = df2['CDR'][df2['M/F'] == "F"]

ss.mannwhitneyu(cdr_m, cdr_f)#This is a non parametric T=test

In [ ]:
sns.lineplot(data=df, x="Day", y="unvaccinated", ci=0, color='blue')
sns.lineplot(data=df, x="Day", y="fully_vaccinated", ci=0,color='red')
sns.lineplot(data=df, x="Day", y="one_booster", ci=0, color='green')
sns.lineplot(data=df, x="Day", y="two_boosters", ci=0)

<h1>Linear Least Squares</h1>

In [ ]:
yD = np.array(dfYd).reshape(-1,1)
xD = np.array(dfXd).reshape(-1,1)
xD.shape, yD.shape

In [ ]:
#Do regression
inter, slope = thinkstats2.LeastSquares(hw[h], hw[w]) # Calculate model it
res = thinkstats2.Residuals(hw[h], hw[w], inter, slope) # Make residual list. Used later, not needed now. 
regLine = thinkstats2.FitLine(hw[h], inter, slope) # Generate line for plotting.

In [ ]:
#Graph it.
#The fitline above calculates the line for us. For an exercise, try to implement a copy of that...
sns.scatterplot(x=hw[h], y=hw[w])
sns.lineplot(x=regLine[0], y=regLine[1], color='red')

In [ ]:
#Plot residuals directly
sns.scatterplot(x=hw[h], y=res)
plt.axhline(0, color="red")

In [ ]:
#Seaborn also has a built in regression plot. 
sns.regplot(x=hw[h], y=hw[w], ci=0)

#Seaborn also has a built in residual plot. 
sns.residplot(x=hw[h], y=hw[w])

Residual graph 

In [ ]:
resNFL = thinkstats2.Residuals(hw2[h2], hw2[w2], int2, slope2)
sns.scatterplot(y=resNFL, x=hw2[h2])
plt.axhline(0, color="red")

regression graph 

In [ ]:
lineNFL = thinkstats2.FitLine(hw2[h2], int2, slope2)
sns.scatterplot(x=hw2[h2], y=hw2[w2])
sns.lineplot(x=lineNFL[0], y=lineNFL[1], color="red")

In [ ]:
def rmseFromResiduals(residuals):
    SSE = 0
    for i in range(len(residuals)):
        SSE += (residuals[i]**2)
    MSE = SSE/len(residuals)
    return math.sqrt(MSE)

## Using Packages for Regression    SKLEARN

In [ ]:
#Setup
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection

In [ ]:
#Deal with data
#ensure that the inputs and outputs are the right shape.
#The -1 means basically "make it one column" in this use. 
x = np.array(hw[h]).reshape(-1,1)
y = np.array(hw[w]).reshape(-1,1)
x.shape, y.shape

In [ ]:
#Generate Model
# Create an instance of a linear regression model and fit it to the data with the fit() function:
#intercept2, slope2 = thinkstats2.LeastSquares(breath["oxigen_per_lit"], breath["peaks_diff"])
model = LinearRegression().fit(x, y) 

In [ ]:
#Get the results of generating the model

#Note: some results come wrapped in an array, that's what the [0]s are for. Remove them to see the true return. 
# Obtain the coefficient of determination by calling the model with the score() function, then print the coefficient of determination to see how good our model is:
r_sq = model.score(x, y)
print('Coefficient of determination (more on this later):', r_sq)
# Print the Intercept:
print('Intercept:', model.intercept_[0])
# Print the Slope:
print('Slope:', model.coef_[0][0]) 

In [ ]:
#Generate predictions. 
#Note - you need to provide the values to predict in an array that is one column wide. 
#Generally you'd make an array of all the things you want to predict and do them en-masse. The [0] is gettign ride of the square brackets. they are used for indexing 
#It grabs the first item of the array 
print("A 183cm tall person is expected to be:", model.predict(np.array(183).reshape(-1,1))[0][0], "kg")

In [ ]:
heightsToPredict = np.array([73, 65, 76, 98, 99])
heightsToPredict = heightsToPredict.reshape(5,1)
heightsToPredict

In [ ]:
modelNFL.predict(heightsToPredict)

In [ ]:
#Graph it
# I'll make a df for ease of use. 
dat = pd.DataFrame(data=np.column_stack((x,y)),columns=['X','Y'])
sns.scatterplot(data=dat, x="X", y="Y")

#Generate the line
inter = model.intercept_[0]
slo = model.coef_[0][0]
inter, slo
lineInf = thinkstats2.FitLine(dat["X"], inter, slo)
sns.lineplot(x=lineInf[0], y=lineInf[1], color="red")

#Probably easier to just use a regplot! 

### Statsmodels

In [ ]:
import statsmodels.api as sm

X2 = sm.add_constant(x)
est = sm.OLS(y, X2) #ordinary least square 
est2 = est.fit()
print(est2.summary())

In [ ]:
toPred = np.array([183, 208, 175])
x_test = sm.add_constant(toPred)
ypred = est2.predict(x_test)
print(ypred)

## R2 - Coefficient of Determination

These two values - RMSE and R2 - are the two primary (mostly - more on this later, along with R2 limitations) metrics that we can use to evaluate a model's accuracy and its predictive value. If we have a high R2 and a (comparatively) low RMSE, we are probably making reliable predictions. 

In [ ]:
def rSquared(yvals, residuals):
    return (1-(thinkstats2.Var(residuals)/thinkstats2.Var(yvals)))

In [ ]:
def rSquared_2(yvals, residuals):
    RSS = 0
    TSS = 0
    ybar = yvals.mean()
    for i in range(len(residuals)):
        RSS += (residuals[i]**2)
        TSS += ((yvals[i]-ybar)**2)
    r2 = 1-(RSS/TSS)
    return r2

In [ ]:
def rmseFromResiduals(residuals):
    SSE = 0
    for i in range(len(residuals)):
        SSE += (residuals[i]**2)
    MSE = SSE/len(residuals)
    return math.sqrt(MSE)

##Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split
ex = ex[["Wrist", "BodyFat"]]
sns.pairplot(ex)


In [ ]:
#Regression data setup
dfYd = ex["BodyFat"]
dfXd = ex["Wrist"]

yD = np.array(dfYd).reshape(-1,1)
xD = np.array(dfXd).reshape(-1,1)
xD.shape, yD.shape

In [ ]:
#Split data
xTrainD,xTestD,yTrainD,yTestD = train_test_split(xD,yD,test_size=.35)

In [ ]:
#Regression
modelD = LinearRegression().fit(xTrainD, yTrainD) 
predsD = modelD.predict(xTestD)

In [ ]:
#RMSE/R2
print("RMSE", mean_squared_error(yTestD, predsD, squared=False))
print("R2", modelD.score(xTestD, yTestD))

Multiple regression



In [ ]:
#Y data is simple, do that first
y = np.array(df["BodyFat"]).reshape(-1,1)
y.shape

In [ ]:
df_ = df.drop(columns={"BodyFat"})
df_.head()

In [ ]:
x = np.array(df_)
x.shape

In [ ]:
#Setup
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection
from sklearn.model_selection import train_test_split

In [ ]:
#Split data
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size=.3)

In [ ]:
#Generate model 
model = LinearRegression().fit(xTrain,yTrain)

In [ ]:
#Get some info on our new regression model
r_sq = model.score(xTest, yTest)
print('R-squared:', r_sq)

In [ ]:
#Get RMSE
tmp = model.predict(xTest)
mean_squared_error(tmp, yTest, squared=False)

In [ ]:
#Get Residuals and picture them in a DF for easy reading. 
tmp1 = pd.DataFrame(yTest, columns={"Y values"})
tmp2 = pd.DataFrame(tmp, columns={"Predictions"})
tmp3 = pd.DataFrame((yTest-tmp), columns={"Residual"})
resFrame = pd.concat([tmp1,tmp2,tmp3], axis=1)
resFrame.head()

In [ ]:
#Examin thresh

posMean, negMean, posSTD, negSTD, posMed, negMed = dPos.mean(), dNeg.mean(), dPos.std(), dNeg.std(), dPos.median(), dNeg.median()
simple_thresh = (posMean + negMean) / 2
simple_thresh

#or

thresh = (posSTD * negMean + negSTD * posMean) / (posSTD + negSTD)
thresh


pos_below_thresh = sum(dPos < thresh)
pos_below_thresh

neg_above_thresh = sum(dNeg > thresh)
neg_above_thresh

pos_overlap = pos_below_thresh / len(dPos)
neg_overlap = neg_above_thresh / len(dNeg)
pos_overlap, neg_overlap

misclassification_rate = (pos_overlap + neg_overlap) / 2
misclassification_rate

#Superiority

sum(x > y for x, y in zip(dPos, dNeg)) / len(dPos)

In [ ]:
def CohenEffectSize(group1, group2):
    """Compute Cohen's d.

    group1: Series or NumPy array
    group2: Series or NumPy array

    returns: float
    """
    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()

    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    d = diff / np.sqrt(pooled_var)
    return d

Plotting cohen effect size 

In [ ]:
def overlap_superiority(control, treatment, n=1000):
    """Estimates overlap and superiority based on a sample.
    
    control: scipy.stats rv object
    treatment: scipy.stats rv object
    n: sample size
    """
    control_sample = control.rvs(n)
    treatment_sample = treatment.rvs(n)
    thresh = (control.mean() + treatment.mean()) / 2
    
    control_above = sum(control_sample > thresh)
    treatment_below = sum(treatment_sample < thresh)
    overlap = (control_above + treatment_below) / n
    
    superiority = (treatment_sample > control_sample).mean()
    return overlap, superiority

In [ ]:
def eval_pdf(rv, num=4):
    mean, std = rv.mean(), rv.std()
    xs = np.linspace(mean - num*std, mean + num*std, 100)
    ys = rv.pdf(xs)
    return xs, ys

In [ ]:
def plot_pdfs(cohen_d=2):
    """Plot PDFs for distributions that differ by some number of stds.
    
    cohen_d: number of standard deviations between the means
    """
    control = ss.norm(0, 1)
    treatment = ss.norm(cohen_d, 1)
    xs, ys = eval_pdf(control)
    plt.fill_between(xs, ys, label='control', color='C1', alpha=0.5)

    xs, ys = eval_pdf(treatment)
    plt.fill_between(xs, ys, label='treatment', color='C0', alpha=0.5)
    
    o, s = overlap_superiority(control, treatment)
    plt.text(0, 0.05, 'overlap ' + str(o))
    plt.text(0, 0.15, 'superiority ' + str(s))
    plt.show()
    #print('overlap', o)
    #print('superiority', s)

In [ ]:
dCohen = CohenEffectSize(dPos, dNeg)
plot_pdfs(dCohen)

Calculating Odds, we can use ratios to make sense of the data 

In [ ]:
#Probability of being overweight
pPos = sum(dPos >= 30)/len(dPos)
pNeg = sum(dNeg >= 30)/len(dNeg)
pPos, pNeg

In [ ]:
#Calculate odds
oddPos = pPos/(1-pPos)
oddNeg = pNeg/(1-pNeg)
oddPos, oddNeg